In [4]:
import pandas as pd
import word_bank.ipynb


ModuleNotFoundError: No module named 'word_bank'

In [5]:
import pandas as pd
from collections import Counter 

recipe_data = pd.read_csv("data/recipes.csv")
print(recipe_data)

column_name = "ingredients"

word_bank = []

measurements = ["cups", "cup", "tbsp", "tablespoons", "tablespoon", "tsp", "teaspoons", "teaspoon",
    "oz", "ounces", "ounce", "pounds", "pound", "lbs", "lb", "grams", "gram", "kg", "kilograms", "kilogram",
    "ml", "milliliters", "milliliter", "liters", "liter", "quarts", "quart", "pints", "pint", "gallons", "gallon"]

fractions = ['½', '¼', '¾', '⅓', '⅔', '⅛', '⅜', '⅝', '⅞']

verbs = ["chopped", "softened", "or to taste", "or as needed", "divided", "melted", "or more to taste", "juiced", 
         "peeled", "drained", "cored", "minced", "beaten", "sliced", "diced", "pitted", "and sliced", "thawed", 
         "cored and chopped", "quartered", "pitted and chopped", "thinly sliced", "mashed", "halved", "seeded", 
         "skinless", "cored and sliced", "pitted and sliced", "finely chopped", "peeled and diced", "peeled and sliced", 
         "pitted and diced", "seeded and diced", "and diced", "or more as needed", "halved and pitted", "lightly beaten", 
         "and chopped", "cut into cubes", "cut into pieces", "crushed", "at room temperature", "cut into chunks", "cored and diced", 
         "and cubed", "room temperature", "cut into -inch cubes", "seeded and chopped", "for garnish", "peeled and chopped", 
         "seeded and cubed", "hulled", "broken into chunks", "peeled and pitted", "and cut into chunks", "cubed", "zested", "shredded",
         "cored and thinly sliced", "peeled and cored", "roughly chopped", "drained with juice reserved", "crumbled", "seeded and minced",
         "toasted", "baked", "stemmed and quarted", "cut into wedges", "coarsely chopped", "peeled and seeds separated", "drained and rinsed", 
         "cored and sliced /-inch thick", "and sliced /-inch thick", "thawed but still cold follow package directions", "cored and sliced / inch thick",
         "heated"]

for index, row in recipe_data.iterrows():
    ingredients = str(row[column_name]).lower()  # Convert to lowercase
    # Remove parentheses by replacing them with empty strings
    ingredients = ingredients.replace("(", "").replace(")", "")
    
    # Remove digits by iterating through characters
    ingredients = ''.join([char if not char.isdigit() else '' for char in ingredients])
    
    for fraction in fractions:
        ingredients = ingredients.replace(fraction, "")

    for verb in verbs:
        ingredients = ingredients.replace(verb, '')
        
    # Split ingredients by commas
    ingredient_list = [ingredient.strip() for ingredient in ingredients.split(',') if ingredient.strip()]
    
    # Filter out measurements and empty strings
    filtered_ingredients = []
    for ingredient in ingredient_list:
        words = ingredient.split()
        filtered_words = [word for word in words if word not in measurements]
        if filtered_words:
            filtered_ingredients.append(" ".join(filtered_words))
   
    for ingredient in filtered_ingredients:
        word_bank.append(ingredient)

    
    
print("This is the word bank", word_bank)



word_frequency = Counter(word_bank)
print("This is the word frequency", word_frequency)
print(len(word_frequency))


      Unnamed: 0                        recipe_name prep_time cook_time  \
0              0           Apple-Cranberry Crostada       NaN       NaN   
1              1          Apple Pie by Grandma Ople   30 mins     1 hrs   
2              2        Sarah's Homemade Applesauce   10 mins   15 mins   
3              3                        Apple Crisp   30 mins   45 mins   
4              4                  Apple Pie Filling   20 mins   20 mins   
...          ...                                ...       ...       ...   
1085        1085                     Date-Nut Balls   15 mins   10 mins   
1086        1086                       Jamaica Cake       NaN       NaN   
1087        1087                     Best Hot Sauce   10 mins   20 mins   
1088        1088               Moist Date Nut Bread   15 mins   50 mins   
1089        1089  Chef John's Chocolate Energy Bars   15 mins       NaN   

         total_time  servings              yield  \
0               NaN         8  6 to 8 - serving

In [11]:
recipe_data = pd.read_csv("data/recipes.csv")

print(recipe_data.head())

   Unnamed: 0                  recipe_name prep_time cook_time     total_time  \
0           0     Apple-Cranberry Crostada       NaN       NaN            NaN   
1           1    Apple Pie by Grandma Ople   30 mins     1 hrs  1 hrs 30 mins   
2           2  Sarah's Homemade Applesauce   10 mins   15 mins        25 mins   
3           3                  Apple Crisp   30 mins   45 mins  1 hrs 15 mins   
4           4            Apple Pie Filling   20 mins   20 mins  2 hrs 40 mins   

   servings              yield  \
0         8  6 to 8 - servings   
1         8       1 9-inch pie   
2         4                NaN   
3        12    1 9x13-inch pan   
4        40      5 9-inch pies   

                                         ingredients  \
0  3 tablespoons butter, 2 pounds Granny Smith ap...   
1  8 small Granny Smith apples, or as needed, ½ c...   
2  4  apples - peeled, cored and chopped, ¾ cup w...   
3  10 cups all-purpose apples, peeled, cored and ...   
4  18 cups thinly sliced app

In [7]:
non_vegan = ['chicken', 'beef', 'lamb', 'duck', 'pork', 'egg', 'milk', 'cheese', 'yogurt', 'butter', 'fish', 'honey', 'salmon', 'cod', 'trout', 'tuna', 'sardine', 'mackerel', 'catfish', 'halibut', 'tilapia', 'mahi-mahi']
non_vegetarian = ['chicken', 'beef', 'lamb', 'duck', 'pork', 'fish', 'salmon', 'cod', 'trout', 'tuna', 'sardine', 'mackerel', 'catfish', 'halibut', 'tilapia', 'mahi-mahi']

def clean_ingredients(ingredients):
    ingredients = str(row[column_name]).lower()  # Convert to lowercase
    # Remove parentheses by replacing them with empty strings
    ingredients = ingredients.replace("(", "").replace(")", "")
    # Remove digits by iterating through characters
    ingredients = ''.join([char if not char.isdigit() else '' for char in ingredients])
    for measurement in measurements:
        ingredients = ingredients.replace(measurement, '')
    for fraction in fractions:
        ingredients = ingredients.replace(fraction, "")
    for verb in verbs:
        ingredients = ingredients.replace(verb, '')
    ingredients = ' '.join(ingredients.split())

    return ingredients

recipe_dict = {}

for index, row in recipe_data.iterrows():
    recipe_name = row['recipe_name']

    ingredients_list = row['ingredients']
    cleaned_ingredients = clean_ingredients(ingredients_list).split(',')
    cleaned_ingredients = [ingredient.strip() for ingredient in cleaned_ingredients if ingredient.strip()]

    recipe_dict[recipe_name] = cleaned_ingredients

ingredients = 'ingredients'
ingredient_keys = [item.lower() for item in ingredients]

def label_diet(ingredient_keys):
    for item in ingredient_keys:
        if item in non_vegetarian:
            return None
    for item in ingredient_keys:    
        if item in non_vegan:
            return "vegetarian"
    return "vegan"
        

diet_labels = []

for index, row in recipe_data.iterrows():
    recipe_name = row['recipe_name']
    cleaned_ingredients = recipe_dict.get(recipe_name, [])
    label = label_diet(cleaned_ingredients)
    diet_labels.append(label)

recipe_data['Dietary Label'] = diet_labels

print(recipe_data[['recipe_name', 'Dietary Label']])


                            recipe_name Dietary Label
0              Apple-Cranberry Crostada    vegetarian
1             Apple Pie by Grandma Ople         vegan
2           Sarah's Homemade Applesauce         vegan
3                           Apple Crisp    vegetarian
4                     Apple Pie Filling         vegan
...                                 ...           ...
1085                     Date-Nut Balls    vegetarian
1086                       Jamaica Cake         vegan
1087                     Best Hot Sauce         vegan
1088               Moist Date Nut Bread    vegetarian
1089  Chef John's Chocolate Energy Bars         vegan

[1090 rows x 2 columns]


In [15]:
def time_to_minutes(time_str):
    
    if not isinstance(time_str, str):
        return None
    
    hours = 0
    minutes = 0
    parts = time_str.lower().split()
    
    for i in range(len(parts)):
        if 'hrs' in parts[i]:
            try:
                hours = int(parts[i - 1])
            except:
                hours = 0
        elif 'mins' in parts[i]:
            try:
                minutes = int(parts[i - 1])
            except:
                minutes = 0
    
    return hours * 60 + minutes

# Create a new column for total time in minutes
recipe_data['Total Time (Minutes)'] = recipe_data['total_time'].apply(time_to_minutes)

# Create a new column for time rating
def get_time_rating(minutes):
    if minutes <= 30:
        return "Short"
    elif minutes <= 120:
        return "Medium"
    elif minutes > 120:
        return "Long"
    else:
        return 'N/A'

recipe_data['Time Rating'] = recipe_data['Total Time (Minutes)'].apply(get_time_rating)

recipe_data


,Unnamed: 0,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src,Total Time (Minutes),Time Rating
0,0,Apple-Cranberry Crostada,NaN,NaN,NaN,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...,NaN,N/A
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...,90.0,Medium
2,2,Sarah's Homemade Applesauce,10 mins,15 mins,25 mins,4,NaN,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,/Side Dish/Applesauce Recipes/,"Total Fat 0g 0%, Sodium 3mg 0%, Total Carbohyd...","Prep Time: 10 mins, Cook Time: 15 mins, Total ...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...,25.0,Short
3,3,Apple Crisp,30 mins,45 mins,1 hrs 15 mins,12,1 9x13-inch pan,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,https://www.allrecipes.com/recipe/12409/apple-...,/Desserts/Crisps and Crumbles Recipes/Apple Cr...,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles...","Prep Time: 30 mins, Cook Time: 45 mins, Total ...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...,75.0,Medium
4,4,Apple Pie Filling,20 mins,20 mins,2 hrs 40 mins,40,5 9-inch pies,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,https://www.allrecipes.com/recipe/12681/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy...","Prep Time: 20 mins, Cook Time: 20 mins, Additi...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...,160.0,Long
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,1085,Date-Nut Balls,15 mins,10 mins,30 mins,48,4 dozen balls,"14 tablespoons butter, 1 cup pitted chopped da...",Melt butter in a saucepan over medium heat; co...,3.8,https://www.allrecipes.com/recipe/240653/date-...,/Desserts/Cookies/Fruit Cookie Recipes/Date/,"Total Fat 5g 6%, Saturated Fat 2g 12%, Cholest...","Prep Time: 15 mins, Cook Time: 10 mins, Additi...",https://www.allrecipes.com/thmb/LXm_15zLymhZ7c...,30.0,Short
1086,1086,Jamaica Cake,NaN,NaN,NaN,14,1 - 13x9 inch pan,"2 cups white sugar, 1 ½ cups vegetable oil, 1 ...","Mix together sugar, vegetable oil, pecans, flo...",4.6,https://www.allrecipes.com/recipe/7509/jamaica...,/Desserts/Fruit Desserts/Banana Dessert Recipes/,"Total Fat 33g 43%, Saturated Fat 4g 21%, Chole...","Servings: 14, Yield: 1 - 13x9 inch pan",https://www.allrecipes.com/thmb/tI7g5xY5-qPV5v...,NaN,N/A
1087,1087,Best Hot Sauce,10 mins,20 mins,30 mins,100,2 cups,"10 fresh hot chile peppers, stems removed, 1 ...","Place peppers, onion, dates, basil, parsley, t...",4.9,https://www.allrecipes.com/recipe/242153/best-...,/Side Dish/Sauces and Condiments/,"Total Fat 1g 1%, Saturated Fat 0g 1%, Sodium 2...","Prep Time: 10 mins, Cook Time: 20 mins, Total ...",https://www.allrecipes.com/thmb/80wUYzFvbycfxx...,30.0,Short
1088,1088,Moist Date Nut Bread,15 mins,50 mins,1 hrs 35 mins,12,1 9x5-inch loaf,"2 ½ cups chopped dates, ¼ cup butter, 1 cup bo...",Preheat the oven to 350 degrees F (175 degrees...,4.5,https://www.allrecipes.com/recipe/51446/moist-...,/Breakfast and Brunch/Breakfast Bread Recipes/,"Total Fat 10g 13%, Saturated Fat 3g 16%, Chole...","Prep Time: 15 mins, Cook Time: 50

In [21]:
sorted_recipes = recipe_data.sort_values(by='rating', ascending = False)

sorted_recipes

,Unnamed: 0,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src,Total Time (Minutes),Time Rating
1077,1077,Pecan Pie Energy Bites,15 mins,10 mins,25 mins,12,12 2-inch balls,"1 cup pecan halves, ⅓ cup rolled oats, 1 cup p...",Preheat the oven to 350 degrees F (175 degrees...,5.0,https://www.allrecipes.com/recipe/8381294/peca...,/Appetizers and Snacks/Snacks/Energy Ball Reci...,"Total Fat 7g 8%, Saturated Fat 1g 3%, Sodium 4...","Prep Time: 15 mins, Cook Time: 10 mins, Total ...",https://www.allrecipes.com/thmb/BDw8F70K9G3J3f...,25.0,Short
1083,1083,Reshteh Polo (Persian Noodle Rice),35 mins,1 hrs 10 mins,3 hrs 35 mins,10,10 servings,"1 cup basmati rice, 3 ½ tablespoons kosher sal...",Place rice in a medium bowl and cover with tep...,5.0,https://www.allrecipes.com/recipe/279194/resht...,/Side Dish/Rice Side Dish Recipes/,"Total Fat 12g 15%, Saturated Fat 4g 22%, Chole...","Prep Time: 35 mins, Cook Time: 1 hrs 10 mins, ...",https://www.allrecipes.com/thmb/2Tqz-HYg7jNg5_...,215.0,Long
1068,1068,Energy Balls without Peanut Butter,20 mins,NaN,2 hrs 20 mins,45,45 balls,"1 cup rolled oats, ½ cup pumpkin seeds, ½ cup ...",Burst oats into a coarse meal in the bowl of a...,5.0,https://www.allrecipes.com/recipe/283043/energ...,/Appetizers and Snacks/Snacks/Energy Ball Reci...,"Total Fat 4g 6%, Saturated Fat 1g 7%, Sodium 5...","Prep Time: 20 mins, Additional Time: 2 hrs, To...",https://www.allrecipes.com/thmb/t_06YCdK084ZN3...,140.0,Long
409,409,Scandinavian Pear Tart,25 mins,25 mins,3 hrs 50 mins,8,1 9-inch tart,"cooking spray, ½ cup white sugar, 3 tablespo...",Preheat the oven to 425 degrees F (220 degrees...,5.0,https://www.allrecipes.com/recipe/276769/scand...,/Desserts/Pies/Tarts/Fruit Tart Recipes/,"Total Fat 17g 22%, Saturated Fat 9g 46%, Chole...","Prep Time: 25 mins, Cook Time: 25 mins, Additi...",https://www.allrecipes.com/thmb/jAN3oDk35AlEvx...,230.0,Long
401,401,Pear Tarte Tatin,20 mins,50 mins,1 hrs 25 mins,8,8 servings,1 (15 ounce) package ready-to-use refrigerated...,"Remove refrigerated pie crust from package, an...",5.0,https://www.allrecipes.com/recipe/269514/pear-...,/Cuisine/European/French/,"Total Fat 22g 28%, Saturated Fat 8g 39%, Chole...","Prep Time: 20 mins, Cook Time: 50 mins, Additi...",https://www.allrecipes.com/thmb/7seKGjAsxcA4QJ...,85.0,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,535,Healthy Apple and Kale Salad,15 mins,NaN,15 mins,3,3 servings,"10 ounces kale, shredded, 2 Honeycrisp apples...",Massage kale with your hands until leaves have...,3.0,https://www.allrecipes.com/recipe/276587/healt...,/Salad/Green Salad Recipes/Kale Salad Recipes/,"Total Fat 10g 13%, Saturated Fat 1g 7%, Sodium...","Prep Time: 15 mins, Total Time: 15 mins, Servi...",https://www.allrecipes.com/thmb/1lsQ5tkXSur6zh...,15.0,Short
344,344,Baked Banana Chips,10 mins,2 hrs,2 hrs 15 mins,2,NaN,"2 just-ripe bananas, sliced in 1/10-inch-thic...",Preheat the oven to 225 degrees F (110 degrees...,2.3,https://www.allrecipes.com/recipe/261342/baked...,/Appetizers and Snacks/Snacks/Snack Chip Recipes/,"Total Fat 0g 1%, Saturated Fat 0g 1%, Sodium 1...","Prep Time: 10 mins, Cook Time: 2 hrs, Addition...",https://www.allrecipes.com/thmb/oku5II_Qq44jrH...,135.0,Long
622,622,Baked Banana Chips,10 mins,2 hrs,2 hrs 15 mins,2,NaN,"2 just-ripe bananas, sliced in 1/10-inch-thic...",Preheat the oven to 225 degrees F (110 degrees...,2.3,https://www.allrecipes.com/recipe/261342/baked...,/Appetizers and Snacks/Snacks/Snack Chip Recipes/,"Total Fat 0g 1%, Saturated Fat 0g 1%, Sodium 1...","Prep Time: 10 mins, Cook Time: 2 hrs, Addition...",https://www.allrecipes.com/thmb/oku5II_Qq44jrH...,135.0,Long
422,422,Dethbert's Pear Pie,30 mins,25 mins,55 mins,16,2 pies,"1 cup all-purpose flour, ½ cup white sugar, ½ ...","Sift flour, sugar, and salt into a mixing bowl...",2.0,https://www.allrecipes.com/recipe/268820/dethb...,/Desserts/Pies/Fruit Pies/Pe